In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os, sys
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
load_dotenv()

SAMPLE_SUBMISSION_ROWS = os.environ.get("SAMPLE_SUBMISSION_ROWS")
TRAINING_DATA = os.environ.get("TRAINING_DATA")

In [3]:
# Obtain training data 

df = pd.read_csv(TRAINING_DATA)

df.head()

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant
0,0,0,NaN,107.215862,26.686594,74.0,3682.75,138.0,111.0,14.0,...,49466.0,65923.0,0.000000,29.180,58.434969,32.500000,86.0,15.856826,168972,152285
1,0,0,NaN,35.552000,50.928261,3.0,1171.35,23.0,17.0,2.0,...,3638.0,7801.0,0.419355,28.465,5.392089,7.000000,125.0,6.998555,212404,39032
2,0,0,NaN,31.623103,48.837872,19.0,2295.38,47.0,42.0,11.0,...,3912.0,12868.0,0.836364,421.500,33.780445,0.000000,180.0,1.753009,225178,7439
3,0,0,NaN,112.277391,NaN,NaN,NaN,NaN,NaN,16.0,...,28919.0,23553.0,0.952381,50.000,37.340085,28.666667,134.0,9.000063,183948,485069
4,1,0,NaN,448.427273,NaN,NaN,NaN,NaN,NaN,5.0,...,1086.0,308.0,0.754386,69.509,77.794164,15.000000,114.0,1.767939,210107,536004


In [4]:
df.shape[0]

12229978

In [5]:
# Compute the correlation matrix between features

correlation_matrix = df.corr()

correlation_matrix

In [6]:
# """Select the features with the greatest correlation to ind_recommended"""

# X_without_ind_reco = df.drop(columns=["ind_recommended", "activation"])
# y_ind_reco = df["ind_recommended"]

# # Calculate correlation coefficients between features and target variable
# correlations = X_without_ind_reco.apply(lambda x: x.corr(y_ind_reco))

# # Sort features based on absolute correlation values
# sorted_features = correlations.abs().sort_values(ascending=False)

# # Select features with highest correlation
# top_features = sorted_features[:50]  # Select top k features, adjust k as needed

# # Extract top features from original dataframe
# df_ind_reco_features = X_without_ind_reco[top_features.index]

# df_ind_reco_features.head()

In [7]:
# """Select the features with the greatest correlation to activation"""

# X_without_activation = df.drop(columns=["activation"])
# y_activation = df["activation"]

# # Calculate correlation coefficients between features and target variable
# correlations = X_without_activation.apply(lambda x: x.corr(y_activation))

# # Sort features based on absolute correlation values
# sorted_features = correlations.abs().sort_values(ascending=False)

# # Select features with highest correlation
# top_features = sorted_features[:50]  # Select top k features, adjust k as needed

# # Extract top features from original dataframe
# df_activation_features = X_without_activation[top_features.index]

# df_activation_features.head()

In [8]:
# ind_reco_features = df_ind_reco_features.columns.to_list()

# activation_features = df_activation_features.columns.to_list()

# print(ind_reco_features)
# print()
# print(activation_features)

In [9]:
# # Visualise the correlation matrix

# plt.figure(figsize=(48, 36))
# sb.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
# plt.title("Correlation Matrix")
# plt.show()

In [10]:
# # Determine which features have the last null/NaN values

# # Count the number of null values in each column
# null_counts = df.isnull().sum()

# # Sort the columns based on the number of null values in ascending order
# sorted_columns = null_counts.sort_values()

# # Print the columns with the least null values
# print("Columns with the least null values:")
# sorted_columns

In [11]:
# # Filter the sorted columns to include only those with no null values
# no_null_columns = sorted_columns[sorted_columns == 0]

# # Print the columns with no null values
# print("Columns with no null values:")
# print(no_null_columns)

In [12]:
# """Remove features with high correlation """

# CORRELATION_THRESHOLD = 0.8

# # Identify highly correlated features
# highly_correlated_features = np.where(correlation_matrix > CORRELATION_THRESHOLD)

# features_to_remove = set()
# for feature1, feature2 in zip(*highly_correlated_features):
#     if feature1 != feature2:
#         if feature1 not in features_to_remove and feature2 not in features_to_remove:
#             features_to_remove.add(feature2)

# # Remove the highly correlated features
# selected_features = [
#     feature for i, feature in enumerate(df.columns) if i not in features_to_remove
# ]
# # selected_df = df[selected_features]

# # print("Selected Features:")
# # selected_df.head()

# ind_reco_features_selected = [f for f in ind_reco_features if f in selected_features]

# print(f"ind_reco_features: {len(ind_reco_features_selected)}")

# activation_features_selected = [f for f in activation_features if f in selected_features]

# print(f"activation_features: {len(activation_features_selected)}")

In [13]:
# """Create a selected_df with the union of the 3 feature groups so that imputation is uniform"""

# core_features = ["customer", "merchant", "ind_recommended", "activation"]

# union_features = list(set().union(ind_reco_features_selected, activation_features_selected, core_features))

# selected_df = df[union_features]

# len(union_features)

In [14]:
# """
# selected_df contains the truncated dataframe after highly-correlated features have been removed.
# Now remove columns with over the specified threshold count of null values.
# """

# # Count the null values in each column
# null_counts = selected_df.isnull().sum()

# NULL_COUNT_THRESHOLD = 3_000_000

# # Filter the DataFrame to preserve only columns with less than `threshold` null values
# filtered_columns = null_counts[null_counts <= NULL_COUNT_THRESHOLD].index
# filtered_df = df[filtered_columns]

# print("Features in filtered DataFrame:")
# features = filtered_df.columns.to_list()
# print(features)
# len(features)

In [15]:
# # Add features that are missing from the Customer Industry Spend and Customer Merchant categories

# filtered_df["customer_merchant_01"] = df["customer_merchant_01"].values
# filtered_df["customer_merchant_02"] = df["customer_merchant_02"].values

# filtered_df.shape[1]

In [16]:
# filtered_df["customer_industry_spend_04"] = df["customer_industry_spend_04"].values
# filtered_df["customer_industry_spend_05"] = df["customer_industry_spend_05"].values

# filtered_df.shape[1]

In [17]:
features = [
    "ind_recommended",
    "activation",
    "customer_spend_01",
    "customer_industry_spend_01",
    "customer_industry_spend_03",
    "customer_spend_02",
    "customer_spend_04",
    "customer_spend_05",
    "customer_spend_06",
    "merchant_spend_06",
    "merchant_spend_07",
    "merchant_profile_01",
    "customer_merchant_03",
    "customer_digital_activity_05",
    "customer_spend_15",
    "customer_digital_activity_16",
    "customer_spend_17",
    "customer_digital_activity_17",
    "customer_digital_activity_03",
    "distance_01",
    "customer_digital_activity_20",
    "distance_02",
    "distance_03",
    "customer_spend_18",
    "customer_spend_19",
    "customer_digital_activity_21",
    "customer_digital_activity_22",
    "distance_04",
    "merchant_profile_02",
    "merchant_spend_09",
    "merchant_profile_03",
    "customer_digital_activity_01",
    "merchant_spend_10",
    "customer_profile_03",
    "customer_digital_activity_02",
    "customer_profile_04",
    "distance_05",
    "customer",
    "merchant",
]

In [18]:
filtered_df = df[features].copy()

In [19]:
# Impute null values with the mean of each column

df_imputed = filtered_df.fillna(filtered_df.median())

In [20]:
# Verify that none of the remaining features have any NULL/NaN values left after interpolation

imputed_null_counts = df_imputed.isnull().sum()

# Filter the columns to include only those with null values
imputed_null_columns = imputed_null_counts[imputed_null_counts > 0]

imputed_null_columns

Series([], dtype: int64)

In [21]:
# """Create the 2 dataframes"""

# filtered_ind_reco_features = [f for f in features if f in ind_reco_features_selected]

# filtered_activation_features = [
#     f for f in features if f in activation_features_selected
# ]

# for core in core_features:
#     if core not in filtered_ind_reco_features:
#         filtered_ind_reco_features.append(core)
#     if core not in filtered_activation_features:
#         filtered_activation_features.append(core)

# print(f"filtered ind reco features: {len(filtered_ind_reco_features)}")
# print(f"filtered activation features: {len(filtered_activation_features)}")

# df_ind_reco = df[filtered_ind_reco_features]  
# df_activation = df[filtered_activation_features]

In [22]:
# """Prepare X and y for ind_reco"""

# X_reco = df_ind_reco.drop(
#     columns=["activation", "customer", "merchant", "ind_recommended"]
# )  # drop customer and merchant too as those are not features
# y_reco = df_ind_reco["ind_recommended"]


In [23]:
# """Prepare X and y for activation"""

# X = df_activation.drop(
#     columns=["activation", "customer", "merchant"]
# )  # drop customer and merchant too as those are not features
# y = df_activation["activation"]


### End of Data Engineering

In [24]:
# Split the data into 5 folds for K-Fold Cross Validation

from sklearn.model_selection import KFold

X = df_imputed.drop(
    columns=["activation", "customer", "merchant"]
)  # drop customer and merchant too as those are not features
y = df_imputed["activation"]

kf = KFold(n_splits=5, shuffle=True, random_state=99)

In [25]:
# Split dataset to predict ind_recommended

X_reco = df_imputed.drop(
    columns=["activation", "customer", "merchant", "ind_recommended"]
)  # drop customer and merchant too as those are not features
y_reco = df_imputed["ind_recommended"]

In [26]:
# # Randomly sample 5_000_000 indices for each k-fold and save them to ensure consistency across models

# # MAX_SAMPLES = 100_000

# K_FOLD_INDICES = []

# for train_index, test_index in kf.split(X, y):
#     # Randomly sample indices for both training and testing
#     # train_index = np.random.choice(
#     #     train_index, size=min(len(train_index), MAX_SAMPLES), replace=False
#     # )
#     # test_index = np.random.choice(
#     #     test_index, size=min(len(test_index), MAX_SAMPLES), replace=False
#     # )

#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#     K_FOLD_INDICES.append([X_train, X_test, y_train, y_test])

In [27]:
# # Randomly sample 5_000_000 indices for each k-fold and save them to ensure consistency across models

# # MAX_SAMPLES = 100_000

# K_FOLD_INDICES_RECO = []

# for train_index, test_index in kf.split(X_reco, y_reco):
#     # Randomly sample indices for both training and testing
#     # train_index = np.random.choice(
#     #     train_index, size=min(len(train_index), MAX_SAMPLES), replace=False
#     # )
#     # test_index = np.random.choice(
#     #     test_index, size=min(len(test_index), MAX_SAMPLES), replace=False
#     # )

#     X_reco_train, X_reco_test = X_reco.iloc[train_index], X_reco.iloc[test_index]
#     y_reco_train, y_reco_test = y_reco.iloc[train_index], y_reco.iloc[test_index]

#     K_FOLD_INDICES_RECO.append([X_reco_train, X_reco_test, y_reco_train, y_reco_test])

### Start of Machine Learning

In [28]:
# """Random Forest Classifier
# Perform Hyperparameter tuning on the RFC with Randomized Search & K - Fold
# """

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import RandomizedSearchCV

# # Define the parameter distribution
# param_dist = {
#     "n_estimators": [50, 100, 200, 300],
#     "max_depth": [None, 10, 20, 30],
#     "min_samples_split": [2, 5, 10],
#     "min_samples_leaf": [1, 2, 4],
#     "bootstrap": [True, False],
# }

# # Instantiate the Random Forest classifier
# rf = RandomForestClassifier()

# # Perform Randomized Search with k-fold cross-validation
# random_search = RandomizedSearchCV(
#     rf, param_dist, n_iter=10, cv=kf, scoring="accuracy", random_state=42
# )
# random_search.fit(X, y)

# # Get the best hyperparameters
# best_params = random_search.best_params_
# print("Best Hyperparameters:", best_params)

# # Train the final model with the best hyperparameters
# model = RandomForestClassifier(**best_params)

In [29]:
# """Random Forest Classifier
# Based on the selected features and whether a merchant was recommended to a customer, use a Random Forest Classifier to predict
# whether a customer will activate.
# """

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score

# model = RandomForestClassifier(n_estimators=100, random_state=42)

# scores = []
# for X_train, X_test, y_train, y_test in K_FOLD_INDICES:

#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)

#     score = accuracy_score(y_test, y_pred)
#     scores.append(score)

# # Display cross-validation scores
# print("Cross-validation scores:", scores)

# # Compute mean cross-validation score
# mean_score = sum(scores) / len(scores)
# print("Mean cross-validation score:", mean_score)

In [30]:
# # Perform k-fold cross-validation
# # Specify the number of folds (k) in the cv parameter
# scores = cross_val_score(clf, X, y, cv=5, scoring="accuracy")

# print("Cross-validation scores:", scores)

# print("Mean accuracy:", scores.mean())

In [31]:
# # Fit the clf model since cross_val_score does not return the fitted cloned models.
# """Logistic Regression
# Based on the selected features and whether a merchant was recommended to a customer, use a Logistic Regression Classifier to predict
# whether a customer will activate.
# """

# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Instantiate Logistic Regression model
# clf = LogisticRegression(max_iter=1000)  # Increase max_iter if needed

# # Split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# # Train the model
# clf.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = clf.predict(X_test)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

In [32]:
# from sklearn.model_selection import train_test_split

# # Split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=58
# )

In [33]:
# from sklearn.model_selection import train_test_split

# # Split data into train and test sets
# X_reco_train, X_reco_test, y_reco_train, y_reco_test = train_test_split(
#     X_reco, y_reco, test_size=0.2, random_state=58
# )

In [34]:
# """Train Random Forest model to predict ind_recommended"""

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score

# # Instantiate Random Forest Classifier
# clf_reco = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# # Fit model
# clf_reco.fit(X_reco_train, y_reco_train)

# # Make predictions
# y_reco_pred = clf_reco.predict(X_reco_test)

# # Evaluate model
# accuracy = accuracy_score(y_reco_test, y_reco_pred)
# print("Accuracy:", accuracy)

In [35]:
# """Train Complement Naive Bayes Classifier to predict ind_recommended"""

# from sklearn.naive_bayes import ComplementNB
# from sklearn.metrics import accuracy_score

# # Instantiate Gaussian Naive Bayes Classifier
# clf_reco = ComplementNB()

# # Fit model
# clf_reco.fit(X_reco_train, y_reco_train)

# # Make predictions
# y_reco_pred = clf_reco.predict(X_reco_test)

# # Evaluate model
# accuracy = accuracy_score(y_reco_test, y_reco_pred)
# print("Accuracy:", accuracy)

In [36]:
# """XGBoost to predict ind_recommended."""

# from xgboost import XGBClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score

# # XGB_BEST_PARAMS = {"learning_rate": 0.1, "max_depth": 5, "n_estimators": 300}

# # Instantiate XGBoost Classifier
# clf_reco = XGBClassifier()

# scores_reco = []
# for X_reco_train, X_reco_test, y_reco_train, y_reco_test in K_FOLD_INDICES_RECO:

#     clf_reco.fit(X_reco_train, y_reco_train)
#     y_reco_pred = clf_reco.predict(X_reco_test)

#     score_reco = accuracy_score(y_reco_test, y_reco_pred)
#     scores_reco.append(score_reco)

# # Display cross-validation scores
# print("Cross-validation scores:", scores_reco)

# # Compute mean cross-validation score
# mean_score = sum(scores_reco) / len(scores_reco)
# print("Mean cross-validation score:", mean_score)

In [37]:
# # Save the random forest classifier model to joblib

# from joblib import dump

# dump(clf_reco, "XGBoost/new_formula/reco.joblib")

In [38]:
# Load the classifier model

from joblib import load

clf_reco = load("XGBoost/new_formula/reco.joblib")

In [39]:
# """XGBoost to predict activation."""

# from xgboost import XGBClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score

# # # Define hyperparameter grid
# # param_grid = {
# #     "max_depth": [3, 5, 10],
# #     "learning_rate": [0.1, 0.01, 0.2],
# #     "n_estimators": [100, 200, 300],
# # }

# # Instantiate XGBoost Classifier
# # xgb = XGBClassifier()

# # # Perform grid search
# # grid_search = GridSearchCV(
# #     estimator=xgb, param_grid=param_grid, scoring="accuracy"
# # )
# # grid_search.fit(X_train, y_train)

# # # Get best hyperparameters
# # best_params = grid_search.best_params_
# # print("Best hyperparameters:", best_params)

# # # Evaluate best model
# # clf = grid_search.best_estimator_
# # y_pred = clf.predict(X_test)
# # accuracy = accuracy_score(y_test, y_pred)
# # print("Accuracy:", accuracy)

# # XGB_BEST_PARAMS = {"learning_rate": 0.1, "max_depth": 5, "n_estimators": 300}

# # Instantiate XGBoost Classifier
# clf = XGBClassifier()

# scores = []
# for X_train, X_test, y_train, y_test in K_FOLD_INDICES:

#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)

#     score = accuracy_score(y_test, y_pred)
#     scores.append(score)

# # Display cross-validation scores
# print("Cross-validation scores:", scores)

# # Compute mean cross-validation score
# mean_score = sum(scores) / len(scores)
# print("Mean cross-validation score:", mean_score)

In [40]:
# """Gaussian Naive Bayes to predict activation."""

# # Instantiate Gaussian Naive Bayes Classifier
# clf = ComplementNB()

# clf.fit(X_train, y_train)

# y_pred = clf.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

In [41]:
# # Save the classifier model to joblib

# from joblib import dump

# dump(clf, "XGBoost/new_formula/activation.joblib")

In [42]:
# Load the classifier model

from joblib import load

clf = load("XGBoost/new_formula/activation.joblib")

In [43]:
# Load the eval dataset

load_dotenv()

EVAL_DATA = os.environ.get("EVAL_DATA")

eval_df = pd.read_csv(EVAL_DATA)

eval_df.head()

,distance_05,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,customer_spend_03,...,merchant_profile_02,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,customer,merchant
0,1.621171,NaN,112.3340,80.5525,9.0,966.63,12.0,10.0,4.0,41.0,...,0.437500,26299.0,4777.0,0.8,33.30,72.268283,7.0,423.0,467915,599167
1,2.441944,NaN,112.3340,NaN,NaN,NaN,NaN,NaN,4.0,41.0,...,0.397059,7122.0,4803.0,0.8,793.29,72.268283,7.0,423.0,467915,686617
2,2.438082,NaN,112.3340,71.1925,3.0,284.77,4.0,4.0,4.0,41.0,...,NaN,7222.0,14860.0,0.8,100.00,72.268283,7.0,423.0,467915,829193
3,2.072182,NaN,112.3340,NaN,NaN,NaN,NaN,NaN,4.0,41.0,...,0.142857,11410.0,11968.0,0.8,252.38,72.268283,7.0,423.0,467915,1077034
4,2.380853,NaN,302.7925,NaN,NaN,NaN,NaN,NaN,3.0,37.0,...,0.100000,1847.0,5842.0,0.8,87.50,72.268283,7.0,423.0,467915,876647


In [44]:
# Only include features used in model training

features_eval = filtered_df.columns.to_list()
# features_eval = list(
#     set().union(filtered_ind_reco_features, filtered_activation_features)
# )

# assert len(features_eval) == 30

# Remove ind_recommended and activation from columns to keep
if "ind_recommended" in features_eval:
    features_eval.remove("ind_recommended")
if "activation" in features_eval:
    features_eval.remove("activation")

eval_df = eval_df.loc[:, features_eval]

# Impute null values with mean

eval_imputed = eval_df.fillna(eval_df.mean())

# Verify that none of the remaining features have any NULL/NaN values left after imputation

eval_imputed_null_counts = eval_imputed.isnull().sum()

# Filter the columns to include only those with null values
eval_imputed_null_columns = eval_imputed_null_counts[eval_imputed_null_counts > 0]

eval_imputed_null_columns

Series([], dtype: int64)

In [45]:
# """Split the eval df into one for ind_reco and one for activation"""

# eval_ind_reco = eval_imputed[
#     [
#         f
#         for f in filtered_ind_reco_features
#         if f != "ind_recommended" and f != "activation"
#     ]
# ]
# eval_activation = eval_imputed[
#     [
#         f
#         for f in filtered_activation_features
#         if f != "ind_recommended" and f != "activation"
#     ]
# ]

In [46]:
# # Run the model on the eval dataset twice, once for ind_recommended = 1 and = 0.

# """Predict ind_recommended values"""

# X_eval = eval_ind_reco.drop(columns=["customer", "merchant"])
    
# # reorder the columns of X_eval to match that of X
# column_order = X_reco.columns
# X_eval = X_eval.reindex(columns=column_order)

# # Drop the ind_recommended ground truth col that appears after reordering
# if "ind_recommended" in X_eval:
#     X_eval.drop(columns=["ind_recommended"], inplace=True)

# # Predict ind_recommended with the clf_reco model
# y_reco_eval = clf_reco.predict_proba(X_eval)[:, 1]

# y_not_reco_eval = clf_reco.predict_proba(X_eval)[:, 0]

# """RECOMMENDED"""
# X_eval["ind_recommended"] = 1

# column_order = X.columns
# X_eval = X_eval.reindex(columns=column_order)

# # Predict probabilities
# y_proba_eval = clf.predict_proba(X_eval)[:, 1]

# # Add the predicted probabilities as a new column to the test dataset
# X_eval["activation_recommended"] = y_proba_eval


# """NOT RECOMMENDED"""
# X_eval["ind_recommended"] = 0

# # We reuse the X_eval but ignore the previously computed activation column
# X_eval_final = X_eval.drop(columns=["activation_recommended"])

# # Predict probabilities
# y_proba_eval = clf.predict_proba(X_eval_final)[:, 1]

# # Add the predicted probabilities as a new column to the test dataset
# X_eval["activation_not_recommended"] = y_proba_eval

# ### Add the predicted ind_recommended to the test dataset
# X_eval["ind_recommended"] = y_reco_eval
# X_eval["ind_not_recommended"] = y_not_reco_eval

In [47]:
# Run the model on the eval dataset twice, once for ind_recommended = 1 and = 0.

"""Predict ind_recommended values"""

X_eval = eval_imputed.drop(columns=["customer", "merchant"])

# reorder the columns of X_eval to match that of X
column_order = X.columns
X_eval = X_eval.reindex(columns=column_order)

# Drop the ind_recommended ground truth col that appears after reordering
X_eval.drop(columns=["ind_recommended"], inplace=True)

# Predict ind_recommended with the clf_reco model
y_reco_eval = clf_reco.predict_proba(X_eval)[:, 1]

y_not_reco_eval = clf_reco.predict_proba(X_eval)[:, 0]

"""RECOMMENDED"""
X_eval["ind_recommended"] = 1

X_eval = X_eval.reindex(columns=column_order)

# Predict probabilities
y_proba_eval = clf.predict_proba(X_eval)[:, 1]

# Add the predicted probabilities as a new column to the test dataset
X_eval["activation_recommended"] = y_proba_eval


"""NOT RECOMMENDED"""
X_eval["ind_recommended"] = 0

# We reuse the X_eval but ignore the previously computed activation column
X_eval_final = X_eval.drop(columns=["activation_recommended"])

# Predict probabilities
y_proba_eval = clf.predict_proba(X_eval_final)[:, 1]

# Add the predicted probabilities as a new column to the test dataset
X_eval["activation_not_recommended"] = y_proba_eval

### Add the predicted ind_recommended to the test dataset
X_eval["ind_recommended"] = y_reco_eval
X_eval["ind_not_recommended"] = y_not_reco_eval

In [116]:
# Add the pred_col to eval

eval_imputed["predicted_score"] = (
    X_eval["activation_recommended"] * X_eval["ind_recommended"] * 3.01
    - X_eval["activation_not_recommended"] * X_eval["ind_not_recommended"]
)

In [49]:
# Generate the pred_score column on the ORIGINAL DATASET
# Eval dataset cannot be used since it is missing the ind_recommended and activation columns
# We will mimic what happens to the eval dataset, except on the original dataset.


def generate_pred_score(X_sampled, y_sampled, X_reco_sampled, y_reco_sampled):
    """X_sampled is the same as X above
    It contains all features except customer, merchant, activation. It contains ind_recommended.

    We want to:
    1. Using purely the features alone, predict probability of ind_recommended being 1
    2. Using purely the features alone, predict probability of ind_recommended being 0
    3. Using features + ind_recommended = 1, predict probability of activation. This is activation given reco.
    4. Using features + ind_recommended = 0, predict probability of activation. This is activation without reco.
    5. Calculate the predicted score.

    y_sampled contains only the activation column.
    """
    X_copy, y_copy, X_reco_copy, y_reco_copy = (
        X_sampled.copy(),
        y_sampled.copy(),
        X_reco_sampled.copy(),
        y_reco_sampled.copy(),
    )  # prevent mutation of original dataframes

    # Predict the probability of ind_recommended being 1
    y_predicted_ind_reco_one = clf_reco.predict_proba(X_reco_copy)[:, 1]

    # Predict the probability of ind_recommended being 0
    y_predicted_ind_reco_zero = clf_reco.predict_proba(X_reco_copy)[:, 0]

    # Predict the probability of activation given ind_recommended = 1
    X_copy["ind_recommended"] = 1
    y_predicted_activation_reco = clf.predict_proba(X_copy)[:, 1]

    # Predict the probability of activation given ind_recommended = 0
    X_copy["ind_recommended"] = 0
    y_predicted_activation_no_reco = clf.predict_proba(X_copy)[:, 1]

    # Restore the original ind_recommended column
    X_copy["ind_recommended"] = y_reco_copy.values

    # Add the predicted probabilities as a new column to the test dataset
    X_copy["predicted_score"] = (
        y_predicted_ind_reco_one * y_predicted_activation_reco
        - y_predicted_ind_reco_zero * y_predicted_activation_no_reco
    )

    # Add the ground truth activation column back to X_sampled
    X_copy["activation"] = y_copy.values

    return X_copy

### Test the scoring function

In [50]:
# # Generate the pred_score column on the ORIGINAL DATASET
# # Eval dataset cannot be used since it is missing the ind_recommended and activation columns
# # We will mimic what happens to the eval dataset, except on the original dataset.


# def generate_pred_score(X_sampled, y_sampled):
#     """X_sampled is the same as X above
#     It contains all features except customer, merchant, activation. It contains ind_recommended.

#     We want to:
#     1. Using purely the features alone, predict probability of ind_recommended being 1
#     2. Using purely the features alone, predict probability of ind_recommended being 0
#     3. Using features + ind_recommended = 1, predict probability of activation. This is activation given reco.
#     4. Using features + ind_recommended = 0, predict probability of activation. This is activation without reco.
#     5. Calculate the predicted score.

#     y_sampled contains only the activation column.
#     """
#     X_copy, y_copy = (
#         X_sampled.copy(),
#         y_sampled.copy(),
#     )  # prevent mutation of original dataframes

#     # # Predict the probability of ind_recommended being 1
#     X_copy_without_ind_reco = X_copy.drop(columns=["ind_recommended"])  # raw features
#     X_actual_ind_reco = X_copy["ind_recommended"].copy()
#     y_predicted_ind_reco_one = clf_reco.predict_proba(X_copy_without_ind_reco)[:, 1]

#     # Predict the probability of ind_recommended being 0
#     y_predicted_ind_reco_zero = clf_reco.predict_proba(X_copy_without_ind_reco)[:, 0]

#     # Predict the probability of activation given ind_recommended = 1
#     X_copy["ind_recommended"] = 1
#     y_predicted_activation_reco = clf.predict_proba(X_copy)[:, 1]

#     # Predict the probability of activation given ind_recommended = 0
#     X_copy["ind_recommended"] = 0
#     y_predicted_activation_no_reco = clf.predict_proba(X_copy)[:, 1]

#     # Restore the original ind_recommended column
#     X_copy["ind_recommended"] = X_actual_ind_reco.values

#     # Add the predicted probabilities as a new column to the test dataset
#     X_copy["predicted_score"] = (
#         y_predicted_ind_reco_one * y_predicted_activation_reco
#         - y_predicted_ind_reco_zero * y_predicted_activation_no_reco
#     )

#     # Add the ground truth activation column back to X_sampled
#     X_copy["activation"] = y_copy.values

#     return X_copy

In [51]:
# Test the scoring function provided

import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

from scoring_code import incr_act_top10

with open("scores.txt", "w+") as f:
    for i in range(1):
        X_sampled_test = generate_pred_score(X, y, X_reco, y_reco)
        
        # Add the customer and merchant columns back on to the dataset
        X_sampled_test["customer"] = df_imputed["customer"].values
        X_sampled_test["merchant"] = df_imputed["merchant"].values

        score = incr_act_top10(input_df=X_sampled_test, pred_col="predicted_score")

        f.write(str(score) + "\n")

   ind_recommended  avg_30d_act
0                0     0.000489
1                1     0.001984


In [92]:
test_features = ["activation_recommended", "activation_not_recommended", "ind_recommended", "ind_not_recommended"]

X_eval[test_features]

,activation_recommended,activation_not_recommended,ind_recommended,ind_not_recommended
0,0.001196,0.001737,0.134526,0.865474
1,0.000325,0.000674,0.139324,0.860676
2,0.001401,0.001928,0.139030,0.860970
3,0.002034,0.003162,0.203926,0.796074
4,0.000781,0.000981,0.229607,0.770393
...,...,...,...,...
12604595,0.003960,0.004547,0.048675,0.951325
12604596,0.000965,0.000866,0.443891,0.556109
12604597,0.001875,0.002453,0.157051,0.842949
12604598,0.000967,0.001095,0.308531,0.691469


### Generate csv for submission

In [117]:
# Extract the submission columns

submission_columns = ["customer", "merchant", "predicted_score"]
submission_df = eval_imputed[submission_columns].copy()

In [118]:
submission_df.head()

,customer,merchant,predicted_score
0,467915,599167,-0.001019
1,467915,686617,-0.000444
2,467915,829193,-0.001074
3,467915,1077034,-0.001269
4,467915,876647,-0.000216


In [119]:
submission_df.shape[0]

12604600

In [120]:
"""Count the number of current submissions so far"""

SUBMISSIONS_DIR = "submissions/"

submissions = os.listdir(SUBMISSIONS_DIR)

submissions_count = len(submissions)

submissions_count

31

In [121]:
submission_df.to_csv(f"submissions/submission_{submissions_count + 1}.csv")

### Submissions
18. XGB, wayne features, emme formula
19. RF, wayne features, emme formula
20. Gaussian NB, wayne features, emme formula
21. Gaussian NB, wayne features, wayne formula
22. XGB, separate features for ind_reco and activation, emme formula

### Optimal value of alpha
1. Submission 25: 0.001162, alpha = 3
2. Submission 26: 0.0003243, alpha = 10
3. Submission 27: 0.0007661, alpha = 6
4. Submission 28: 0.0009583, alpha = 4
5. Submission 29: 0.00106, alpha = 3.5
6. Submission 30: 0.0011244, alpha = 3.25
7. Submission 31: 0.0011528, alpha = 3.1